In [5]:
# https://github.com/So-ham/Movie-Recommendation-System/blob/main/movie-recommendation.ipynb

# https://towardsdatascience.com/how-did-we-build-book-recommender-systems-in-an-hour-part-2-k-nearest-neighbors-and-matrix-c04b3c2ef55c

# _Imports and Opening Datasets_


In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

In [7]:
animes_path = '../data/AnimeList.csv'
users_path = '../data/UserList.csv'
reviews_path = '../data/UserAnimeList.parquet'

In [8]:
reviews_df = pd.read_parquet(reviews_path, columns=['username', 'anime_id', 'my_score'])
animes_df = pd.read_csv(animes_path)

In [9]:
reviews_df = reviews_df.merge(animes_df, on='anime_id', how='left')

# Animes_df Data Exploration and Preparation


In [10]:
reviews_df['user_id'] = reviews_df.groupby("username").ngroup()


In [11]:
reviews_df.shape

(80076112, 34)

In [12]:
reviews_df = reviews_df.head(8000000)
reviews_df.head()

,username,anime_id,my_score,title,title_english,title_japanese,title_synonyms,image_url,type,source,...,premiered,broadcast,related,producer,licensor,studio,genre,opening_theme,ending_theme,user_id
0,karthiga,21,9,One Piece,One Piece,ONE PIECE,OP,https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,...,Fall 1999,Sundays at 09:30 (JST),"{'Adaptation': [{'mal_id': 13, 'type': 'manga'...","Fuji TV, TAP, Shueisha","Funimation, 4Kids Entertainment",Toei Animation,"Action, Adventure, Comedy, Super Power, Drama,...","['#01: ""We Are! (ウィーアー!)"" by Hiroshi Kitadani ...","['#01: ""memories"" by Maki Otsuki (eps 1-30)', ...",222757
1,karthiga,59,7,Chobits,Chobits,ちょびっツ,NaN,https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,...,Spring 2002,Wednesdays at 02:20 (JST),"{'Adaptation': [{'mal_id': 107, 'type': 'manga...","TBS, Pioneer LDC","Funimation, Geneon Entertainment USA",Madhouse,"Sci-Fi, Comedy, Drama, Romance, Ecchi, Seinen","['""Let Me Be With You"" by ROUND TABLE feat. Ni...","['#1: ""Raison d&#039;Être"" by Rie Tanaka (eps ...",222757
2,karthiga,74,7,Gakuen Alice,Gakuen Alice,学園アリス,"Campus Alice, Alice Academy",https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,...,Fall 2004,Unknown,"{'Adaptation': [{'mal_id': 320, 'type': 'manga...","Aniplex, Sogo Vision",Nozomi Entertainment,Group TAC,"Comedy, School, Shoujo, Super Power","['""Pikapika no Taiyou"" by Ueda, Kana']","['""Shiawase no Niji"" by Ueda Kana & Kugimiya R...",222757
3,karthiga,120,7,Fruits Basket,Fruits Basket,フルーツバスケット,Furuba,https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,...,Summer 2001,Thursdays at 18:00 (JST),"{'Adaptation': [{'mal_id': 102, 'type': 'manga...","TV Tokyo, Nihon Ad Systems, King Records",Funimation,Studio Deen,"Slice of Life, Comedy, Drama, Romance, Fantasy...","['""For Fruits Basket (For フルーツバスケット)"" by Ritsu...","['#1: ""Chiisana Inori (小さな祈り)"" by Ritsuko Okaz...",222757
4,karthiga,178,7,Ultra Maniac,Ultramaniac - Magical Girl,ウルトラマニアック,NaN,https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,...,Spring 2003,Tuesdays at 22:00 (JST),"{'Adaptation': [{'mal_id': 956, 'type': 'manga...","WAO World, Studio Jack","Discotek Media, Geneon Entertainment USA",Production Reed,"Magic, Comedy, Romance, School, Shoujo","['""Kagami no Naka"" by can/goo']","['""Hitotsu=Unmei Kyoudoutai"" by can/goo']",222757


In [13]:
reviews_df[reviews_df['anime_id']==21]

,username,anime_id,my_score,title,title_english,title_japanese,title_synonyms,image_url,type,source,...,premiered,broadcast,related,producer,licensor,studio,genre,opening_theme,ending_theme,user_id
0,karthiga,21,9,One Piece,One Piece,ONE PIECE,OP,https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,...,Fall 1999,Sundays at 09:30 (JST),"{'Adaptation': [{'mal_id': 13, 'type': 'manga'...","Fuji TV, TAP, Shueisha","Funimation, 4Kids Entertainment",Toei Animation,"Action, Adventure, Comedy, Super Power, Drama,...","['#01: ""We Are! (ウィーアー!)"" by Hiroshi Kitadani ...","['#01: ""memories"" by Maki Otsuki (eps 1-30)', ...",222757
58,RedvelvetDaisuki,21,0,One Piece,One Piece,ONE PIECE,OP,https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,...,Fall 1999,Sundays at 09:30 (JST),"{'Adaptation': [{'mal_id': 13, 'type': 'manga'...","Fuji TV, TAP, Shueisha","Funimation, 4Kids Entertainment",Toei Animation,"Action, Adventure, Comedy, Super Power, Drama,...","['#01: ""We Are! (ウィーアー!)"" by Hiroshi Kitadani ...","['#01: ""memories"" by Maki Otsuki (eps 1-30)', ...",127798
761,Damonashu,21,10,One Piece,One Piece,ONE PIECE,OP,https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,...,Fall 1999,Sundays at 09:30 (JST),"{'Adaptation': [{'mal_id': 13, 'type': 'manga'...","Fuji TV, TAP, Shueisha","Funimation, 4Kids Entertainment",Toei Animation,"Action, Adventure, Comedy, Super Power, Drama,...","['#01: ""We Are! (ウィーアー!)"" by Hiroshi Kitadani ...","['#01: ""memories"" by Maki Otsuki (eps 1-30)', ...",35503
1110,bskai,21,8,One Piece,One Piece,ONE PIECE,OP,https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,...,Fall 1999,Sundays at 09:30 (JST),"{'Adaptation': [{'mal_id': 13, 'type': 'manga'...","Fuji TV, TAP, Shueisha","Funimation, 4Kids Entertainment",Toei Animation,"Action, Adventure, Comedy, Super Power, Drama,...","['#01: ""We Are! (ウィーアー!)"" by Hiroshi Kitadani ...","['#01: ""memories"" by Maki Otsuki (eps 1-30)', ...",190000
1563,shuzzable,21,10,One Piece,One Piece,ONE PIECE,OP,https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,...,Fall 1999,Sundays at 09:30 (JST),"{'Adaptation': [{'mal_id': 13, 'type': 'manga'...","Fuji TV, TAP, Shueisha","Funimation, 4Kids Entertainment",Toei Animation,"Action, Adventure, Comedy, Super Power, Drama,...","['#01: ""We Are! (ウィーアー!)"" by Hiroshi Kitadani ...","['#01: ""memories"" by Maki Otsuki (eps 1-30)', ...",261363
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7994548,NDBruttezza,21,7,One Piece,One Piece,ONE PIECE,OP,https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,...,Fall 1999,Sundays at 09:30 (JST),"{'Adaptation': [{'mal_id': 13, 'type': 'manga'...","Fuji TV, TAP, Shueisha","Funimation, 4Kids Entertainment",Toei Animation,"Action, Adventure, Comedy, Super Power, Drama,...","['#01: ""We Are! (ウィーアー!)"" by Hiroshi Kitadani ...","['#01: ""memories"" by Maki Otsuki (eps 1-30)', ...",107754
7996554,hijikata_,21,7,One Piece,One Piece,ONE PIECE,OP,https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,...,Fall 1999,Sundays at 09:30 (JST),"{'Adaptation': [{'mal_id': 13, 'type': 'manga'...","Fuji TV, TAP, Shueisha","Funimation, 4Kids Entertainment",Toei Animation,"Action, Adventure, Comedy, Super Power, Drama,...","['#01: ""We Are! (ウィーアー!)"" by Hiroshi Kitadani ...","['#01: ""memories"" by Maki Otsuki (eps 1-30)', ...",213129
7997405,lolichanx,21,0,One Piece,One Piece,ONE PIECE,OP,https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,...,Fall 1999,Sundays at 09:30 (JST),"{'Adaptation': [{'mal_id': 13, 'type': 'manga'...","Fuji TV, TAP, Shueisha","Funimation, 4Kids Entertainment",Toei Animation,"Action, Adventure, Comedy, Super Power, Drama,...","['#01: ""We Are! (ウィーアー!)"" by Hiroshi Kitadani ...","['#01: ""memories"" by Maki Otsuki (eps 1-30)', ...",231514
7997711,love1314eva,21,9,One Piece,One Piece,ONE PIECE,OP,https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,...,Fall 1999,Sundays at 09:30 (JST),"{'Adaptation': 

In [52]:
final_dataset = reviews_df.pivot(index='anime_id', columns='user_id', values='my_score')
final_dataset.head()

user_id,15,23,28,43,50,72,82,91,116,137,...,282958,282965,282967,282970,282980,282989,283012,283020,283026,283032
anime_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,0.0,NaN,0.0,0.0,NaN,10.0,NaN,NaN,...,10.0,NaN,8.0,NaN,8.0,0.0,NaN,0.0,NaN,9.0
5,NaN,NaN,NaN,NaN,NaN,0.0,NaN,8.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0
6,NaN,NaN,0.0,NaN,0.0,0.0,NaN,10.0,NaN,NaN,...,8.0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,9.0
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,0.0
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
final_dataset.fillna(0,inplace=True)
final_dataset.head()

user_id,15,23,28,43,50,72,82,91,116,137,...,282958,282965,282967,282970,282980,282989,283012,283020,283026,283032
anime_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,...,10.0,0.0,8.0,0.0,8.0,0.0,0.0,0.0,0.0,9.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,...,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [54]:
csr_data = csr_matrix(final_dataset.values)

In [55]:
from sklearn.neighbors import NearestNeighbors
model_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=20, n_jobs=-1)

In [56]:
knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=20, n_jobs=-1)


In [57]:
knn.fit(final_dataset)

NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1, n_neighbors=20)

In [96]:
final_dataset.reset_index(inplace=True)
final_dataset

user_id,anime_id,15,23,28,43,50,72,82,91,116,...,282958,282965,282967,282970,282980,282989,283012,283020,283026,283032
0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,...,10.0,0.0,8.0,0.0,8.0,0.0,0.0,0.0,0.0,9.0
1,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,...,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0
3,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14450,37877,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14451,37878,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14452,37886,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14453,37887,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [97]:
def get_movie_recommendation(movie_name):
    n_movies_to_reccomend = 10
    anime_list = reviews_df[reviews_df['title_english'].str.contains(movie_name, na=False)]  
    if len(anime_list):        
        anime_id= anime_list.iloc[0]['anime_id']
        #anime_id = final_dataset[final_dataset['anime_id'] == anime_id].index[0]
        anime_id = final_dataset[final_dataset.index==anime_id].index[0]
        distances , indices = knn.kneighbors(csr_data[anime_id], n_neighbors=n_movies_to_reccomend+1)    
        rec_movie_indices = sorted(list(zip(indices.squeeze().tolist(),distances.squeeze().tolist())),\
                               key=lambda x: x[1])[:0:-1]
        
        recommend_frame = []
        
        for val in rec_movie_indices:
            anime_id = final_dataset.iloc[val[0]]['anime_id']
            idx = reviews_df[reviews_df['anime_id'] == anime_id].index
            recommend_frame.append({'Title':reviews_df.iloc[idx]['title'].values[0],'Distance':val[1]})
        df = pd.DataFrame(recommend_frame,index=range(1,n_movies_to_reccomend+1))
        return df
    
    else:
        
        return "No movies found. Please check your input"


In [102]:
get_movie_recommendation('Samurai Champloo')

,Title,Distance
1,Mahoromatic 2,0.743350
2,Puni Puni☆Poemii,0.739441
3,Ichigo Mashimaro,0.735130
4,Abenobashi Mahou☆Shoutengai,0.734669
5,Bokusatsu Tenshi Dokuro-chan,0.733142
6,Mahoromatic: Automatic Maiden,0.731900
7,Renkin San-kyuu Magical? Pokaan,0.731331
8,Heppoko Jikken Animation Excel♥Saga,0.723027
9,Paniponi Dash!,0.722787
10,Azumanga Daioh,0.710169
